<a href="https://colab.research.google.com/github/daigo0927/blog/blob/feature%2Fmulti-gpu/multi-gpu-training/train_tf_multiworker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Wed May 12 14:35:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

In [5]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

# tf.config.experimental.set_virtual_device_configuration(
#     gpus[0],    
#     [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096),
#      tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]
# )
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(logical_gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


In [32]:
def build_network(image_size, n_classes):
    image = layers.Input([*image_size, 3], dtype=tf.float32, name='input')

    effnet = EfficientNetB0(include_top=False,
                            weights='imagenet',
                            pooling='avg')
    feature = effnet(image)
    logit = layers.Dense(n_classes)(feature)
    model = tf.keras.Model(inputs=image, outputs=logit)
    return model

In [36]:
strategy = tf.distribute.MirroredStrategy(logical_gpus)
print(strategy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [37]:
SEED = 42
IMAGE_SIZE = (128, 128)
EPOCHS = 3
BATCH_SIZE = 32
LEARNING_RATE = 0.001
N_CLASSES = 120
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [38]:
def preprocess(image, label):
    image = tf.image.resize_with_pad(image, *IMAGE_SIZE)
    image = tf.cast(image, dtype=tf.float32)
    return image, label

def augment(image, label):
    image = tf.image.random_brightness(image, 0.3, SEED)
    image = tf.image.random_contrast(image, 0.5, 1.0, SEED)
    image = tf.image.random_flip_left_right(image, SEED)
    return image, label

(ds_train, ds_val), info = tfds.load('stanford_dogs', split=['train', 'test'], as_supervised=True, with_info=True)
print(info)    

ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.shuffle(buffer_size=len(ds_train), seed=SEED, reshuffle_each_iteration=True)
ds_train = ds_train.batch(BATCH_SIZE).prefetch(buffer_size=BATCH_SIZE)

ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(buffer_size=BATCH_SIZE)

tfds.core.DatasetInfo(
    name='stanford_dogs',
    version=0.2.0,
    description='The Stanford Dogs dataset contains images of 120 breeds of dogs from around
the world. This dataset has been built using images and annotation from
ImageNet for the task of fine-grained image categorization. There are
20,580 images, out of which 12,000 are used for training and 8580 for
testing. Class labels and bounding box annotations are provided
for all the 12,000 images.',
    homepage='http://vision.stanford.edu/aditya86/ImageNetDogs/main.html',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'image/filename': Text(shape=(), dtype=tf.string),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=120),
        'objects': Sequence({
            'bbox': BBoxFeature(shape=(4,), dtype=tf.float32),
        }),
    }),
    total_num_examples=20580,
    splits={
        'test': 8580,
        'train': 12000,
    },
    supervised_keys=('image'

In [40]:
with strategy.scope():
    model = build_network(IMAGE_SIZE, N_CLASSES)
    optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = tf.keras.metrics.SparseCategoricalAccuracy()

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [41]:
model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_val,
    steps_per_epoch=len(ds_train),
    validation_steps=len(ds_val)
)

Epoch 1/10
375/375 [==============================] - ETA: 0s - loss: 3.4706 - sparse_categorical_accuracy: 0.2100

KeyboardInterrupt: ignored